##Gathering the Dataset

In [54]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
from selenium.webdriver.common.by import By

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [59]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import csv

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


In [60]:
import csv

fields = ['question', 'percent','id'] 
filename = "wouldyourather_seperated.csv"

In [61]:
with open(filename, 'w') as csvfile:   
  writer = csv.DictWriter(csvfile, fieldnames = fields) 
  writer.writeheader() 

  for i in range(3000):
    wd.get(f"http://either.io/{i}")

    url1 = f'/vote/{i}/1'
    url2 = f'/vote/{i}/2'

    if len(wd.find_elements_by_xpath('//a[@href="'+url1+'"]')) >= 1:

      option1 = wd.find_element_by_xpath('//a[@href="'+url1+'"]')
  
      option2 = wd.find_element_by_xpath('//a[@href="'+url2+'"]')

      percents = wd.find_elements_by_class_name("percentage")
      option_percent1 = int(percents[0].get_attribute("innerHTML").replace("<span>","").replace("</span>","").replace("%",""))
      option_percent2 = 100-option_percent1

      question1 = option1.get_attribute("text").replace("\n","")
      question2 = option2.get_attribute("text").replace("\n","")

      myobj1 = [{'question': question1 , 'percent': option_percent1, 'id': i}]
      myobj2 = [{'question': question2 , 'percent': option_percent2, 'id': i}]
    
      writer.writerows(myobj1)
      writer.writerows(myobj2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead


##Preprocessing Data

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import numpy.random as random
import math
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [3]:
data_directory = '/content/drive/MyDrive/CS/Datasets/wyr.csv'
db = pd.read_csv(data_directory)

In [4]:
db.head()
print(db.shape)

(1374, 3)


In [5]:
df = db[db.percent > 0]
print(df.shape)

(974, 3)


In [6]:
result = []
percentcol = df["percent"]

percent_list = percentcol.tolist()

for i in percent_list:
  if i <=50:
    result.append(1)
  else:
    result.append(0)
print(result)

df["result"] = result
df.head()

[1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,question,percent,id,result
0,Run 26 miles or Swim 5 miles,50,3,1
1,Your partner have a homosexual affair or Your ...,59,4,0
2,Have a Pepsi or Have a Coke,35,5,1
3,Watch only dramas for the rest of your life or...,14,6,1
4,Burn to death or Drown,31,7,1


In [7]:
tokenizer = Tokenizer(nb_words=2500, lower=True,split=' ')
tokenizer.fit_on_texts(df['question'].values)

X = tokenizer.texts_to_sequences(df['question'].values)
X = pad_sequences(X)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/text.py:180: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Keras Imports

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Embedding, Dense
from keras.callbacks import EarlyStopping
from keras.constraints import maxnorm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

##BOW/Feed Forward on Seperated Dataset

In [62]:
data_directory_seperate = '/content/drive/MyDrive/CS/Datasets/wyr_seperated.csv'
db2 = pd.read_csv(data_directory_seperate)

In [67]:
result = []

db2 = db2[db2.percent > 0]
db2 = db2[db2.percent < 100]
percentcol = db2["percent"]

percent_list = percentcol.tolist()

for i in percent_list:
  if i <=50:
    result.append(0)
  else:
    result.append(1)

print(result)

db2["result"] = result
db2.head()

[0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 

,question,percent,id,result
0,Run 26 miles,50,3,0
1,Swim 5 miles,50,3,0
2,Your partner have a homosexual affair,59,4,1
3,Your partner have a heterosexual affair,41,4,0
4,Have a Pepsi,35,5,0


In [68]:
print(db2.shape)

(1962, 4)


BAG OF WORDS

In [152]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [153]:
db2.head()

,question,percent,id,result
0,Run 26 miles,50,3,0
1,Swim 5 miles,50,3,0
2,Your partner have a homosexual affair,59,4,1
3,Your partner have a heterosexual affair,41,4,0
4,Have a Pepsi,35,5,0


In [154]:
print("Samples per class: {}".format(np.bincount(db2.result)))

Samples per class: [1001  961]


In [155]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(db2.question,db2.result,test_size = 0.2,random_state=20, stratify=db2.result)

In [156]:
vectorizer = CountVectorizer()

In [157]:
print("Samples per class: {}".format(np.bincount(y_train)))
print("Samples per class: {}".format(np.bincount(y_test)))

Samples per class: [800 769]
Samples per class: [201 192]


In [158]:
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [159]:
feature_names = vectorizer.get_feature_names()
print("Number of features: {}".format(len(feature_names)))
print("First 20 features:\n{}".format(feature_names[:20]))
print("Features 19500 to 19530:\n{}".format(feature_names[19500:19530]))
print("Every 2000th feature:\n{}".format(feature_names[::2000]))

Number of features: 2373
First 20 features:
['000', '0wl', '10', '100', '11', '12', '120', '1337', '13th', '15', '150', '16', '18', '1963', '1970', '1989', '1st', '20', '200', '2008']
Features 19500 to 19530:
[]
Every 2000th feature:
['000', 'story']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [160]:
vectorizer.vocabulary_

{'have': 943,
 'met': 1319,
 'jesus': 1097,
 'store': 1998,
 'all': 90,
 'your': 2365,
 'photos': 1576,
 'on': 1472,
 'facebook': 729,
 'date': 546,
 'topanga': 2150,
 'from': 829,
 'boy': 287,
 'meets': 1308,
 'world': 2344,
 'drool': 624,
 'abnormally': 47,
 'in': 1038,
 'public': 1656,
 'ferrari': 759,
 'work': 2341,
 'under': 2202,
 'captain': 361,
 'kirk': 1146,
 'live': 1230,
 'homeless': 988,
 'shelter': 1832,
 'with': 2327,
 'internet': 1057,
 'access': 54,
 'extremely': 720,
 'muscular': 1385,
 'legs': 1196,
 'spend': 1939,
 'day': 549,
 'the': 2101,
 'north': 1446,
 'pole': 1611,
 'wear': 2286,
 'winter': 2324,
 'clothes': 445,
 'summer': 2024,
 'burn': 327,
 'to': 2133,
 'death': 555,
 'desert': 561,
 'only': 1478,
 'listen': 1227,
 'banjo': 191,
 'music': 1387,
 'and': 110,
 'let': 1200,
 'sibling': 1855,
 'die': 577,
 'be': 205,
 'forced': 799,
 'watch': 2278,
 'tv': 2187,
 'time': 2130,
 '100': 3,
 'million': 1333,
 'dollars': 601,
 'but': 335,
 'chance': 395,
 'of': 1466

In [161]:
i = 100
j = 10
words = vectorizer.get_feature_names()[i:i+10]
pd.DataFrame(X_train[j:j+7, i:i+10].todense(), columns=words)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,always,amazing,amazon,amelia,american,amish,amount,amputate,an,ancient
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0


In [162]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.constraints import maxnorm
from sklearn.preprocessing import MinMaxScaler

num_cols = X_train.shape[1]

model = Sequential()

model.add(Dense(100, input_shape=(num_cols,), activation='relu'))
model.add(Dense(600, activation='relu'))
model.add(Dense(600, activation='relu'))
model.add(Dense(400, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 100)               237400    
                                                                 
 dense_55 (Dense)            (None, 600)               60600     
                                                                 
 dense_56 (Dense)            (None, 600)               360600    
                                                                 
 dense_57 (Dense)            (None, 400)               240400    
                                                                 
 dense_58 (Dense)            (None, 100)               40100     
                                                                 
 dense_59 (Dense)            (None, 1)                 101       
                                                                 
Total params: 939,201
Trainable params: 939,201
Non-tr

In [164]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(X_train.shape)
print(y_train.shape)
print(X_train)
print(y_train)

early_stopping_monitor = EarlyStopping(patience=40)

model.fit(X_train, y_train, epochs=2000, batch_size=128, callbacks=[early_stopping_monitor])
model.evaluate(X_test, y_test)

(1569, 2373)
(1569,)
  (0, 943)	1
  (0, 1319)	1
  (0, 1097)	1
  (1, 1998)	1
  (1, 90)	1
  (1, 2365)	1
  (1, 1576)	1
  (1, 1472)	1
  (1, 729)	1
  (2, 546)	1
  (2, 2150)	1
  (2, 829)	1
  (2, 287)	1
  (2, 1308)	1
  (2, 2344)	1
  (3, 624)	1
  (3, 47)	1
  (3, 1038)	1
  (3, 1656)	1
  (4, 759)	1
  (5, 2341)	1
  (5, 2202)	1
  (5, 361)	1
  (5, 1146)	1
  (6, 1038)	1
  :	:
  (1564, 152)	1
  (1564, 179)	1
  (1564, 1744)	1
  (1564, 2371)	1
  (1564, 1063)	1
  (1564, 302)	1
  (1564, 975)	1
  (1564, 1142)	1
  (1565, 2101)	1
  (1565, 100)	1
  (1565, 2085)	1
  (1565, 2179)	1
  (1566, 2133)	1
  (1566, 1156)	1
  (1566, 1448)	1
  (1566, 1693)	1
  (1566, 1011)	1
  (1567, 1193)	1
  (1567, 293)	1
  (1568, 2365)	1
  (1568, 797)	1
  (1568, 1448)	1
  (1568, 2080)	1
  (1568, 1361)	1
  (1568, 311)	1
512     1
1082    1
260     0
1366    0
1875    1
       ..
987     0
154     1
160     0
1095    1
2140    1
Name: result, Length: 1569, dtype: int64
Epoch 1/2000


InvalidArgumentError: ignored

This method was found not to work in this Bag of Words notebook using preliminary statistical analysis: https://colab.research.google.com/drive/1dLEHoeumoQtTtdHOTRW6NvGHlCA5Xk6E?authuser=1#scrollTo=B4La9V2Tluue&uniqifier=1

OLD DATA CLASSIFICATION

In [46]:
optionlist2 = []
optionid = []
for index, row in db2.iterrows():
  optionx = str(row["option_1"].replace("\n",""))
  row["option_2"] = row["option_2"].replace("\n","")
  print(optionx)
  row["option_1"] = optionx
  optionlist2.append(row["option_2"])
  optionid.append(row["id"])

print(optionlist2)

Be swept up in a hurricane 
Overdose on every drug at the same time 
End world hunger 
Go back in time and correct the single biggest mistake of your life 
Stick your cut up hand into a bowl of vinegar 
Be hunted by people 
Marry a 10 year old, and have a faithful relationship while receiving $100,000 per year 
The Amazon jungle 
Be captured by a wild tribe in the jungles 
Have a lightsaber (Star Wars) 
Kill 5 people with a knife 
Eat a rancid mixture of rotten egg and old fish 
Only play Risk 
Superbowl 
Burp extremely loudly every 30 seconds 
Eat a Butterfinger 
Be near deaf 
Kill yourself 
Swallow three jawbreakers 
Be locked in an NYC studio apt with 400 pugs for a year 
Be extremely paranoid 
Listen to only old Broadway musical songs 
Beat a tiger in a fight who ate a man 
Army crawl everywhere 
Always speak in pig-latin 
Always have a sore throat 
Be an only child 
Reddit 
Be stupid and have a brilliant kid 
Live to be 120 and accomplish nothing notable 
Personal chauffeur 
Never

##LSTM on Joint Dataset

In [18]:
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = X.shape[1]))
model.add(Dropout(.2))
model.add(LSTM(lstm_out))
model.add(Embedding(2500, embed_dim, input_length = lstm_out))
model.add(LSTM(lstm_out))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 44, 128)           320000    
                                                                 
 dropout_3 (Dropout)         (None, 44, 128)           0         
                                                                 
 lstm_10 (LSTM)              (None, 200)               263200    
                                                                 
 embedding_10 (Embedding)    (None, 200, 128)          320000    
                                                                 
 lstm_11 (LSTM)              (None, 200)               263200    
                                                                 
 dense_4 (Dense)             (None, 2)                 402       
                                                                 
Total params: 1,166,802
Trainable params: 1,166,802
No

In [19]:
y = pd.get_dummies(df['result']).values
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 36)

model.fit(X_train, y_train, batch_size =batch_size, epochs = 20)

model.evaluate(X_test, y_test)

Epoch 1/20
25/25 [==============================] - 15s 481ms/step - loss: 0.6985 - accuracy: 0.4737
Epoch 2/20
25/25 [==============================] - 12s 480ms/step - loss: 0.6943 - accuracy: 0.4994
Epoch 3/20
25/25 [==============================] - 12s 483ms/step - loss: 0.6935 - accuracy: 0.4596
Epoch 4/20
25/25 [==============================] - 12s 477ms/step - loss: 0.6939 - accuracy: 0.4891
Epoch 5/20
25/25 [==============================] - 12s 485ms/step - loss: 0.6934 - accuracy: 0.4994
Epoch 6/20
25/25 [==============================] - 12s 485ms/step - loss: 0.6937 - accuracy: 0.4955
Epoch 7/20
25/25 [==============================] - 12s 478ms/step - loss: 0.6942 - accuracy: 0.4942
Epoch 8/20
25/25 [==============================] - 12s 485ms/step - loss: 0.6939 - accuracy: 0.4711
Epoch 9/20
25/25 [==============================] - 12s 484ms/step - loss: 0.6933 - accuracy: 0.4827
Epoch 10/20
25/25 [==============================] - 12s 481ms/step - loss: 0.6934 - accura

[0.6931540966033936, 0.482051283121109]